In [1]:
test_file = "1_cols_100_mb"

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LargeRowBenchMark").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/27 12:45:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Parquet

In [3]:
df = spark.read.parquet(f"data/{test_file}.parquet")
df.createOrReplaceTempView("t")
df

DataFrame[uid: bigint, name: string, col_0: array<double>]

24/09/27 12:46:01 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
df.collect()

In [ ]:
from pyspark.sql.functions import array_max

df.select(array_max("col_0")).collect()

In [4]:
from pyspark.sql.functions import array_sort

df.select(array_sort("col_0").alias('sorted_col_0')).collect()

24/09/27 12:46:39 ERROR Executor: Exception in task 3.0 in stage 1.0 (TID 4)/ 8]
java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.sql.catalyst.expressions.codegen.BufferHolder.grow(BufferHolder.java:80)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeArrayWriter.initialize(UnsafeArrayWriter.java:71)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:389)
	at org.apache.spark.sql.execution.SparkPlan$$Lambda/0x000000012ed8ca80.apply(Unknown Source)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:893)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:893)
	at org.apache.spark.rdd.RDD$$Lambda/0x000

ConnectionRefusedError: [Errno 61] Connection refused

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/yuchuanhuang/Github/large-row/venv/lib/python3.12/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yuchuanhuang/Github/large-row/venv/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yuchuanhuang/Github/large-row/venv/lib/python3.12/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [ ]:
import pyspark.sql.functions as F

df.select(F.aggregate("col_0", F.lit(0.0), lambda acc, x: acc + x).alias("total")).collect()

In [ ]:
from pyspark.sql.types import DoubleType

def get_second_max(arr):
    if arr is None or len(arr) < 2:
        return None
    sorted_arr = sorted(arr, reverse=True)
    if len(sorted_arr) < 2:
        return None
    return sorted_arr[1]

spark.udf.register("get_second_max", get_second_max, DoubleType())
spark.sql("select get_second_max(col_0) from t").collect()

# CSV

In [ ]:
df = spark.read.csv(f"data/{test_file}.csv", header=True)
df

In [ ]:
# convert col_0 to an array of doubles
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import ArrayType, DoubleType

df = df.withColumn("col_0", from_json(col("col_0"), ArrayType(DoubleType())))
df.createOrReplaceTempView("t")
df

In [ ]:
df.collect()

In [ ]:
df.select(array_max("col_0")).collect()

In [ ]:
df.select(array_sort("col_0").alias('sorted_col_0')).collect()

In [ ]:
df.select(F.aggregate("col_0", F.lit(0.0), lambda acc, x: acc + x).alias("total")).collect()

In [ ]:
spark.sql("select get_second_max(col_0) from t").collect()

# Stop Spark

In [5]:
spark.stop()